In [1]:
from faiss.swigfaiss import Index
from langame.langame_client import LangameClient
import fire
import torch
import datetime
import faiss
import glob
from sentence_transformers import SentenceTransformer
import numpy as np
import logging
from autofaiss import build_index
import os
from collections import defaultdict


In [9]:
sentence_embeddings_model_name = "sentence-transformers/LaBSE"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

sentence_embeddings_model = SentenceTransformer(sentence_embeddings_model_name, device=device)
new_memes = []
new_memes_embeddings = []

with open("../data/common_2021_12_15.txt", "r") as f:
    print(f"Building embeddings for {len(f.readlines())} new memes")
    for i, line in enumerate(f):
        if i % 1000 == 0:
            print(f"Built embeddings for {i} new memes")
        line = line.strip()
        # Check that the format is correct
        # i.e. [topics,] ### [sentence]
        splitted = line.split("###")
        if line and len(splitted) == 2:
            topics, sentence = splitted
            print(topics, sentence)
            new_memes.append(line)
            new_memes_embeddings.append(
                sentence_embeddings_model.encode(
                    sentence.strip(), show_progress_bar=False
                )
            )
# new_memes_embeddings = np.array(new_memes_embeddings)
# index, index_infos = build_index(
#     new_memes_embeddings,
#     save_on_disk=False,
# )

Building embeddings for 91571 new memes


In [8]:
new_memes_embeddings.shape

(0,)

In [10]:
indexes = index.search(sentence_embeddings_model.encode(["What is the purpose of life"], show_progress_bar=False), k=10)[1]
for e in indexes[0]:
    print(new_memes[e])

philosophy,ice breaker ### What is the meaning of life?
philosophy,ice breaker ### What is the meaning of life?
philosophy ### What is the meaning of life ?
philosophy,ice breaker ### What is the meaning of life?
life,philosophy,mind ### What is the meaning of life?
philosophy,ice breaker ### What is the meaning of life?
philosophy,ice breaker ### What is the meaning of life?
life,ice breaker ### What is your life goal?
philosophy,ice breaker ### What are the meaning of life?
ice breaker ### What is a life goal of yours?
